# Scraping Linkedin Data 

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time

In [2]:
EMAIL = "YOUR_EMAIL_ID"
PASSWORD = "PASSWORD"
def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    return driver

In [3]:
def login_to_linkedin(driver):
    driver.get("https://www.linkedin.com/login")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(EMAIL)
    driver.find_element(By.ID, "password").send_keys(PASSWORD + Keys.RETURN)


In [4]:
def scrape_profiles(driver, profile_urls):
    data = []
    for url in profile_urls:
        try:
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            name = driver.find_element(By.CSS_SELECTOR, "h1.text-heading-xlarge").text
            positions = driver.find_elements(By.CSS_SELECTOR, ".experience-item__title")
            companies = driver.find_elements(By.CSS_SELECTOR, ".experience-item__company-name")
            industries = driver.find_elements(By.CSS_SELECTOR, ".experience-item__industry")

            for pos, comp, ind in zip(positions, companies, industries):
                data.append({
                    "Name": name,
                    "Job Title": pos.text,
                    "Company": comp.text,
                    "Industry": ind.text
                })

        except TimeoutException:
            print(f"Timeout while loading: {url}")
        except NoSuchElementException:
            print(f"Missing data on: {url}")
    return data


In [5]:
def save_to_csv(data, filename="linkedin_profiles.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

In [6]:
def main():
    driver = init_driver()
    try:
        login_to_linkedin(driver)

       
        profile_urls = [
            "https://www.linkedin.com/in/srushti-admuthe-b03285240/"
        ]

        data = scrape_profiles(driver, profile_urls)
        save_to_csv(data)

    finally:
        driver.quit()

In [8]:
if __name__ == "__main__":
    main()

Missing data on: https://www.linkedin.com/in/srushti-admuthe-b03285240/
Data saved to linkedin_profiles.csv
